In [58]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [59]:
import gym
import random

In [60]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [61]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
        
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:22.0
Episode:2 Score:35.0
Episode:3 Score:44.0
Episode:4 Score:11.0
Episode:5 Score:24.0
Episode:6 Score:19.0
Episode:7 Score:12.0
Episode:8 Score:26.0
Episode:9 Score:24.0
Episode:10 Score:43.0


In [62]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [63]:
def build_model(states, actio):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [64]:
model = build_model(states, actions)

In [65]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_18 (Dense)             (None, 24)                120       
_________________________________________________________________
dense_19 (Dense)             (None, 24)                600       
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [66]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [67]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn



In [68]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 184s 18ms/step - reward: 1.0000
110 episodes - episode_reward: 89.827 [10.000, 200.000] - loss: 2.816 - mae: 19.512 - mean_q: 39.518

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 143s 14ms/step - reward: 1.0000
52 episodes - episode_reward: 191.019 [153.000, 200.000] - loss: 6.712 - mae: 39.906 - mean_q: 80.503

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 114s 11ms/step - reward: 1.0000
54 episodes - episode_reward: 185.944 [139.000, 200.000] - loss: 6.856 - mae: 45.644 - mean_q: 91.968

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 100s 10ms/step - reward: 1.0000
54 episodes - episode_reward: 184.963 [142.000, 200.000] - loss: 5.016 - mae: 43.442 - mean_q: 87.352

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 120s 12ms/step - reward: 1

In [69]:
scores =  dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 196.000, steps: 196
Episode 2: reward: 193.000, steps: 193
Episode 3: reward: 184.000, steps: 184
Episode 4: reward: 177.000, steps: 177
Episode 5: reward: 180.000, steps: 180
Episode 6: reward: 177.000, steps: 177
Episode 7: reward: 187.000, steps: 187
Episode 8: reward: 190.000, steps: 190
Episode 9: reward: 190.000, steps: 190
Episode 10: reward: 182.000, steps: 182
Episode 11: reward: 177.000, steps: 177
Episode 12: reward: 186.000, steps: 186
Episode 13: reward: 171.000, steps: 171
Episode 14: reward: 175.000, steps: 175
Episode 15: reward: 175.000, steps: 175
Episode 16: reward: 188.000, steps: 188
Episode 17: reward: 181.000, steps: 181
Episode 18: reward: 182.000, steps: 182
Episode 19: reward: 174.000, steps: 174
Episode 20: reward: 185.000, steps: 185
Episode 21: reward: 178.000, steps: 178
Episode 22: reward: 180.000, steps: 180
Episode 23: reward: 190.000, steps: 190
Episode 24: reward: 191.000, steps: 191
Episode 25: reward: 

In [70]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 182.000, steps: 182
Episode 2: reward: 196.000, steps: 196
Episode 3: reward: 183.000, steps: 183
Episode 4: reward: 187.000, steps: 187
Episode 5: reward: 177.000, steps: 177
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 188.000, steps: 188
Episode 9: reward: 187.000, steps: 187
Episode 10: reward: 197.000, steps: 197
Episode 11: reward: 195.000, steps: 195
Episode 12: reward: 186.000, steps: 186
Episode 13: reward: 182.000, steps: 182
Episode 14: reward: 174.000, steps: 174
Episode 15: reward: 180.000, steps: 180


In [71]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [76]:
del model
del dqn
del env

In [77]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [78]:
dqn.load_weights('dqn_weights.h5f')

In [79]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 180.000, steps: 180
Episode 2: reward: 181.000, steps: 181
Episode 3: reward: 186.000, steps: 186
Episode 4: reward: 173.000, steps: 173
Episode 5: reward: 191.000, steps: 191
